In [2]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
import zipfile
from pathlib import Path

from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt


start = time.time()
#print(start)
print(datetime.datetime.now())

#pdf_file = 'Lindbergh Lawns Tract  BL 1,2,3.pdf'
zip_file = 'test.zip'

zip_file_path = '/home/vimalkumar/Documents/stewart/zip'
zip_path = '/home/vimalkumar/Documents/stewart/Pdf files'

os.makedirs(os.path.join(zip_path,os.path.splitext(zip_file)[0]))

with zipfile.ZipFile(os.path.join(zip_file_path,zip_file),"r") as zip_ref:
    #print(zip_ref)
    zip_ref.extractall(os.path.join(zip_path,os.path.splitext(zip_file)[0]))
    names = zip_ref.namelist()
    print(names)
    zip_folder = names[0][:-1]
    print(zip_folder)
    
file_path = '/home/vimalkumar/Documents/stewart/Pdf files'
images_path = '/home/vimalkumar/Documents/stewart/Images'
#excel_path = '/home/vimalkumar/Documents/stewart/excel_output'
excel_path = '/home/vimalkumar/Documents/stewart/zip'

os.makedirs(os.path.join(images_path,os.path.splitext(zip_file)[0]))
#os.makedirs(os.path.join(excel_path,os.path.splitext(zip_file)[0]))

pdf_name = []
file_name = []
inception_result = []
vgg_result = []
ocr_result = []
vgg19_result = []

for pdf_file in sorted(os.listdir(os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder))):
    if pdf_file.endswith('.pdf')or pdf_file.endswith('.PDF')or pdf_file.endswith('.Pdf'):
        print(pdf_file)
        in_path = os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder,pdf_file)
        out_path = os.path.join(images_path,os.path.splitext(zip_file)[0],
                                str(os.path.splitext(pdf_file)[0]))

        os.makedirs(out_path)
        
        file_size=Path(os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder,pdf_file)).stat().st_size
        
        if file_size != 0:
            doc = fitz.open(in_path)

            print('No of Images: ',len(doc))



            for i in range(len(doc)):
                doc = fitz.open(in_path)
                page = doc.load_page(i)  # number of page
                pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
                f_name = str(i+1000 + 1)       
                pix.pil_save(os.path.join(out_path,f_name+'.jpeg'))

            dic = {0:'Blank',1:'Not Blank'}

            inception_model = keras.models.load_model('stewart_inception_model.h5')
            vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')
            vgg19_model = keras.models.load_model('stewart_vgg19_model_1.h5')


#             pdf_name = []
#             file_name = []
#             inception_result = []
#             vgg_result = []
#             ocr_result = []
#             vgg19_result = []


            cnt = 1

            for im in sorted(os.listdir(out_path)):
                pdf_name.append(pdf_file)
                file_name.append(cnt)
                #print(cnt)
                print(cnt,'  ',im)
                cnt = cnt + 1
                img = image.load_img(os.path.join(out_path,im),
                                 target_size=(224,224))
                #plt.imshow(img)
                #plt.show()

                x = image.img_to_array(img)
                x = x/255
                x = np.expand_dims(x,axis=0)
                images = np.vstack([x])
                val = inception_model.predict(images)
                #print(val)

                a=np.argmax(val,axis=1)
                output_inception = dic[a[0]]
                #print(output_inception)
                #file_name.append(im)

                #image_read = cv2.imread(os.path.join(in_path,im),0)
                pil_read = Image.open(os.path.join(out_path,im))
                pil_read_gray = ImageOps.grayscale(pil_read)
                img_rgb = np.array(pil_read_gray)


                _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

                nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
                sizes = stats[1:, -1] #get CC_STAT_AREA component
                img2 = np.zeros((labels.shape), np.uint8)

                for i in range(0, nlabels - 1):
                    if sizes[i] >= 50:   #filter small dotted regions
                        img2[labels == i + 1] = 255

                res = cv2.bitwise_not(img2)
            #     data = Image.fromarray(res)
            #     enh_con = ImageEnhance.Contrast(data)
            #     contrast = 3.0
            #     image_contrasted = enh_con.enhance(contrast)
                image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
                image_data_lower = " ".join(image_data.splitlines()).lower()

                stats = ImageStat.Stat(pil_read)
                #print(stats)

                y = []
                for band,name in enumerate(pil_read.getbands()):
                    x=sum(stats.extrema[band])
                    y.append(x)
                if sum(y)==765:
                    #print('good')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray)
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)

                else:
                    #print('dark')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
                    #print(image_data_1_lower)



                if output_inception == 'Blank':
                    inception_result.append(output_inception)
                else:


                    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                        inception_result.append('New Document')
                    else:
                        inception_result.append('Not Blank')


                val_1 = vgg_model.predict(images)
                #print(val_1)

                a_1=np.argmax(val_1,axis=1)
                output_vgg = dic[a_1[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg == 'Blank':
                    vgg_result.append(output_vgg)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg > 80 or fuzz_ratio_vgg_1 > 80:
                        vgg_result.append('New Document')
                    else:
                        vgg_result.append('Not Blank')


                val_2 = vgg19_model.predict(images)
                #print(val_1)

                a_2=np.argmax(val_2,axis=1)
                output_vgg19 = dic[a_2[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg19 == 'Blank':
                    vgg19_result.append(output_vgg19)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg19 = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg19_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg19 > 80 or fuzz_ratio_vgg19_1 > 80:
                        vgg19_result.append('New Document')
                    else:
                        vgg19_result.append('Not Blank')



                fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                    ocr_result.append('New Document')

                elif len(image_data_1_lower) < 15:
                    ocr_result.append('Blank')

                else:
                    ocr_result.append('Not Blank')


df = pd.DataFrame({'File_Name':pdf_name,'Page_Number':file_name,'Inception_result':inception_result,
                               'VGG_result':vgg_result,'VGG19_result':vgg19_result,
                               'OCR_Result':ocr_result})
#display(df)
df['Confidence']=(df['Inception_result']==df['VGG_result'])&(df['Inception_result']==df['VGG19_result'])&(df['Inception_result']==df['OCR_Result'])

df['status'] = df['Inception_result']
df['Confidence'] = df['Confidence'].replace(True,'Yes')
df['Confidence'] = df['Confidence'].replace(False,'No')

pd.set_option('display.max_rows',1000)
display(df)

final_df = df[['File_Name','Page_Number','status','Confidence']]
display(final_df)

final_df.to_excel(os.path.join(excel_path,
                               str(os.path.splitext(zip_file)[0])+'.xlsx'),index=False)            

end = time.time()

print(end - start)

    



2022-02-18 17:03:00.341470
['test/', 'test/Batch 2.pdf', 'test/Batch 4.pdf', 'test/Batch 14.pdf']
test
Batch 14.pdf
No of Images:  88


2022-02-18 17:03:09.783624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-18 17:03:09.783688: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-18 17:03:09.783724: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2022-02-18 17:03:09.784222: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1    1001.jpeg


2022-02-18 17:03:23.629811: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg
25    1025.jpeg
26    1026.jpeg
27    1027.jpeg
28    1028.jpeg
29    1029.jpeg
30    1030.jpeg
31    1031.jpeg
32    1032.jpeg
33    1033.jpeg
34    1034.jpeg
35    1035.jpeg
36    1036.jpeg
37    1037.jpeg
38    1038.jpeg
39    1039.jpeg
40    1040.jpeg
41    1041.jpeg
42    1042.jpeg
43    1043.jpeg
44    1044.jpeg
45    1045.jpeg
46    1046.jpeg
47    1047.jpeg
48    1048.jpeg
49    1049.jpeg
50    1050.jpeg
51    1051.jpeg
52    1052.jpeg
53    1053.jpeg
54    1054.jpeg
55    1055.jpeg
56    1056.jpeg
57    1057.jpeg
58    1058.jpeg
59    1059.jpeg
60    1060.jpeg
61    1061.jpeg
62    1062.jpeg
63    1063.jpeg
64    1064.jpeg


,File_Name,Page_Number,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,Batch 14.pdf,1,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
1,Batch 14.pdf,2,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
2,Batch 14.pdf,3,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
3,Batch 14.pdf,4,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
4,Batch 14.pdf,5,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
5,Batch 14.pdf,6,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
6,Batch 14.pdf,7,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
7,Batch 14.pdf,8,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
8,Batch 14.pdf,9,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
9,Batch 14.pdf,10,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


,File_Name,Page_Number,status,Confidence
0,Batch 14.pdf,1,Not Blank,Yes
1,Batch 14.pdf,2,Not Blank,Yes
2,Batch 14.pdf,3,Not Blank,Yes
3,Batch 14.pdf,4,Not Blank,Yes
4,Batch 14.pdf,5,Not Blank,Yes
5,Batch 14.pdf,6,Not Blank,Yes
6,Batch 14.pdf,7,Not Blank,Yes
7,Batch 14.pdf,8,Not Blank,Yes
8,Batch 14.pdf,9,Not Blank,Yes
9,Batch 14.pdf,10,Not Blank,Yes


923.8382604122162


In [19]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
from datetime import datetime

import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
import zipfile
from pathlib import Path

from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt


start = time.time()
#print(start)
print(datetime.now())

#pdf_file = 'Lindbergh Lawns Tract  BL 1,2,3.pdf'
zip_file = 'zip_test.zip'

date_time = datetime.now()
date_time = str(date_time).replace(' ','_')[:-7]
#print(str(date_time).replace(' ','_')[:-7])
#print("Datetime from timestamp:", date_time)

zip_file_path = '/home/vimalkumar/Documents/stewart/1/Zip'
zip_path = '/home/vimalkumar/Documents/stewart/1/Pdf files'

os.makedirs(os.path.join(zip_path,os.path.splitext(zip_file)[0]+str(date_time)))

with zipfile.ZipFile(os.path.join(zip_file_path,zip_file),"r") as zip_ref:
    #print(zip_ref)
    zip_ref.extractall(os.path.join(zip_path,os.path.splitext(zip_file)[0]+str(date_time)))
    names = zip_ref.namelist()
    print(names)
    zip_folder = names[0][:-1]
    print(zip_folder)
    
file_path = '/home/vimalkumar/Documents/stewart/1/Pdf files'
images_path = '/home/vimalkumar/Documents/stewart/1/Images'
#excel_path = '/home/vimalkumar/Documents/stewart/excel_output'
excel_path = '/home/vimalkumar/Documents/stewart/1/excel'

os.makedirs(os.path.join(images_path,os.path.splitext(zip_file)[0]+str(date_time)))
os.makedirs(os.path.join(excel_path,os.path.splitext(zip_file)[0]+str(date_time)))


pdf_name = []
file_name = []
inception_result = []
vgg_result = []
ocr_result = []
vgg19_result = []

for pdf_file in sorted(os.listdir(os.path.join(file_path,os.path.splitext(zip_file)[0]+str(date_time),zip_folder))):
    if pdf_file.endswith('.pdf')or pdf_file.endswith('.PDF')or pdf_file.endswith('.Pdf'):
        print(pdf_file)
        in_path = os.path.join(file_path,os.path.splitext(zip_file)[0]+str(date_time),zip_folder,pdf_file)
        out_path = os.path.join(images_path,os.path.splitext(zip_file)[0]+str(date_time),
                                str(os.path.splitext(pdf_file)[0]))

        os.makedirs(out_path)
        
        file_size=Path(os.path.join(file_path,os.path.splitext(zip_file)[0]+str(date_time),zip_folder,pdf_file)).stat().st_size
        
        if file_size != 0:
            doc = fitz.open(in_path)

            print('No of Images: ',len(doc))



            for i in range(len(doc)):
                doc = fitz.open(in_path)
                page = doc.load_page(i)  # number of page
                pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
                f_name = str(i+1000 + 1)       
                pix.pil_save(os.path.join(out_path,f_name+'.jpeg'))





2022-02-21 14:07:53.644510
['zip_test/', 'zip_test/1507-1513.pdf', 'zip_test/Batch 4_Pt2.pdf']
zip_test
1507-1513.pdf
No of Images:  4
Batch 4_Pt2.pdf


In [ ]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
import zipfile
from pathlib import Path

from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt


start = time.time()
#print(start)
print(datetime.datetime.now())

#pdf_file = 'Lindbergh Lawns Tract  BL 1,2,3.pdf'
zip_file = 'zip_test.zip'



zip_file_path = '/home/vimalkumar/Documents/stewart/zip'
zip_path = '/home/vimalkumar/Documents/stewart/Pdf files'

os.makedirs(os.path.join(zip_path,os.path.splitext(zip_file)[0]))

with zipfile.ZipFile(os.path.join(zip_file_path,zip_file),"r") as zip_ref:
    #print(zip_ref)
    zip_ref.extractall(os.path.join(zip_path,os.path.splitext(zip_file)[0]))
    names = zip_ref.namelist()
    print(names)
    zip_folder = names[0][:-1]
    print(zip_folder)
    
file_path = '/home/vimalkumar/Documents/stewart/Pdf files'
images_path = '/home/vimalkumar/Documents/stewart/Images'
#excel_path = '/home/vimalkumar/Documents/stewart/excel_output'
excel_path = '/home/vimalkumar/Documents/stewart/zip'

os.makedirs(os.path.join(images_path,os.path.splitext(zip_file)[0]))
#os.makedirs(os.path.join(excel_path,os.path.splitext(zip_file)[0]))

pdf_name = []
file_name = []
inception_result = []
vgg_result = []
ocr_result = []
vgg19_result = []

for pdf_file in sorted(os.listdir(os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder))):
    if pdf_file.endswith('.pdf')or pdf_file.endswith('.PDF')or pdf_file.endswith('.Pdf'):
        print(pdf_file)
        in_path = os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder,pdf_file)
        out_path = os.path.join(images_path,os.path.splitext(zip_file)[0],
                                str(os.path.splitext(pdf_file)[0]))

        os.makedirs(out_path)
        
        file_size=Path(os.path.join(file_path,os.path.splitext(zip_file)[0],zip_folder,pdf_file)).stat().st_size
        
        if file_size != 0:
            doc = fitz.open(in_path)

            print('No of Images: ',len(doc))



            for i in range(len(doc)):
                doc = fitz.open(in_path)
                page = doc.load_page(i)  # number of page
                pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
                f_name = str(i+1000 + 1)       
                pix.pil_save(os.path.join(out_path,f_name+'.jpeg'))

            dic = {0:'Blank',1:'Not Blank'}

            inception_model = keras.models.load_model('stewart_inception_model.h5')
            vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')
            vgg19_model = keras.models.load_model('stewart_vgg19_model_1.h5')


#             pdf_name = []
#             file_name = []
#             inception_result = []
#             vgg_result = []
#             ocr_result = []
#             vgg19_result = []


            cnt = 1

            for im in sorted(os.listdir(out_path)):
                pdf_name.append(pdf_file)
                file_name.append(cnt)
                #print(cnt)
                print(cnt,'  ',im)
                cnt = cnt + 1
                img = image.load_img(os.path.join(out_path,im),
                                 target_size=(224,224))
                #plt.imshow(img)
                #plt.show()

                x = image.img_to_array(img)
                x = x/255
                x = np.expand_dims(x,axis=0)
                images = np.vstack([x])
                val = inception_model.predict(images)
                #print(val)

                a=np.argmax(val,axis=1)
                output_inception = dic[a[0]]
                #print(output_inception)
                #file_name.append(im)

                #image_read = cv2.imread(os.path.join(in_path,im),0)
                pil_read = Image.open(os.path.join(out_path,im))
                pil_read_gray = ImageOps.grayscale(pil_read)
                img_rgb = np.array(pil_read_gray)


                _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

                nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
                sizes = stats[1:, -1] #get CC_STAT_AREA component
                img2 = np.zeros((labels.shape), np.uint8)

                for i in range(0, nlabels - 1):
                    if sizes[i] >= 50:   #filter small dotted regions
                        img2[labels == i + 1] = 255

                res = cv2.bitwise_not(img2)
            #     data = Image.fromarray(res)
            #     enh_con = ImageEnhance.Contrast(data)
            #     contrast = 3.0
            #     image_contrasted = enh_con.enhance(contrast)
                image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
                image_data_lower = " ".join(image_data.splitlines()).lower()

                stats = ImageStat.Stat(pil_read)
                #print(stats)

                y = []
                for band,name in enumerate(pil_read.getbands()):
                    x=sum(stats.extrema[band])
                    y.append(x)
                if sum(y)==765:
                    #print('good')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray)
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)

                else:
                    #print('dark')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
                    #print(image_data_1_lower)



                if output_inception == 'Blank':
                    inception_result.append(output_inception)
                else:


                    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                        inception_result.append('New Document')
                    else:
                        inception_result.append('Not Blank')


                val_1 = vgg_model.predict(images)
                #print(val_1)

                a_1=np.argmax(val_1,axis=1)
                output_vgg = dic[a_1[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg == 'Blank':
                    vgg_result.append(output_vgg)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg > 80 or fuzz_ratio_vgg_1 > 80:
                        vgg_result.append('New Document')
                    else:
                        vgg_result.append('Not Blank')


                val_2 = vgg19_model.predict(images)
                #print(val_1)

                a_2=np.argmax(val_2,axis=1)
                output_vgg19 = dic[a_2[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg19 == 'Blank':
                    vgg19_result.append(output_vgg19)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg19 = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg19_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg19 > 80 or fuzz_ratio_vgg19_1 > 80:
                        vgg19_result.append('New Document')
                    else:
                        vgg19_result.append('Not Blank')



                fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                    ocr_result.append('New Document')

                elif len(image_data_1_lower) < 15:
                    ocr_result.append('Blank')

                else:
                    ocr_result.append('Not Blank')


df = pd.DataFrame({'File_Name':pdf_name,'Page_Number':file_name,'Inception_result':inception_result,
                               'VGG_result':vgg_result,'VGG19_result':vgg19_result,
                               'OCR_Result':ocr_result})
#display(df)
df['Confidence']=(df['Inception_result']==df['VGG_result'])&(df['Inception_result']==df['VGG19_result'])&(df['Inception_result']==df['OCR_Result'])

df['status'] = df['Inception_result']
df['Confidence'] = df['Confidence'].replace(True,'Yes')
df['Confidence'] = df['Confidence'].replace(False,'No')

pd.set_option('display.max_rows',1000)
display(df)

final_df = df[['File_Name','Page_Number','status','Confidence']]
display(final_df)

final_df.to_excel(os.path.join(excel_path,
                               str(os.path.splitext(zip_file)[0])+'.xlsx'),index=False)            

end = time.time()

print(end - start)

    



In [7]:
from datetime import datetime
 
# Getting Datetime from timestamp
date_time = datetime.now()
print(str(date_time).replace(' ','_')[:-7])
print("Datetime from timestamp:", date_time)

2022-02-21_12:10:09
Datetime from timestamp: 2022-02-21 12:10:09.117056


In [54]:
date = datetime.date(datetime.now())
time = datetime.time(datetime.now())
os.makedirs(os.path.join('/home/vimalkumar/Documents/stewart/1',str(date),str(time)[:-7]))


/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2022-03-18 10:16:37.790747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 10:16:37.790772: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Single Folder path model

In [15]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
import zipfile
from pathlib import Path

from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

start = time.time()

file_path = '/home/vimalkumar/Documents/stewart/pdf'

folder_path_name = os.path.basename(file_path)
print(folder_path_name)



images_path = '/home/vimalkumar/Documents/stewart/2/images'
#excel_path = '/home/vimalkumar/Documents/stewart/excel_output'
excel_path = '/home/vimalkumar/Documents/stewart/pdf'

os.makedirs(os.path.join(images_path,folder_path_name))
#os.makedirs(os.path.join(excel_path,os.path.splitext(zip_file)[0]))

pdf_name = []
file_name = []
inception_result = []
vgg_result = []
ocr_result = []
vgg19_result = []

for pdf_file in sorted(os.listdir(os.path.join(file_path))):
    pdf_file_extension = pdf_file.lower()
    if pdf_file_extension.endswith('.pdf'):
        print(pdf_file)
        in_path = os.path.join(file_path,pdf_file)
        out_path = os.path.join(images_path,folder_path_name,
                                str(os.path.splitext(pdf_file)[0]))

        os.makedirs(out_path)
        
        file_size=Path(os.path.join(file_path,pdf_file)).stat().st_size
        
        if file_size != 0:
            doc = fitz.open(in_path)

            print('No of Images: ',len(doc))



            for i in range(len(doc)):
                doc = fitz.open(in_path)
                page = doc.load_page(i)  # number of page
                pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
                f_name = str(i+1000 + 1)       
                pix.pil_save(os.path.join(out_path,f_name+'.jpeg'))

            dic = {0:'Blank',1:'Not Blank'}

            inception_model = keras.models.load_model('stewart_inception_model.h5')
            vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')
            vgg19_model = keras.models.load_model('stewart_vgg19_model_1.h5')


#             pdf_name = []
#             file_name = []
#             inception_result = []
#             vgg_result = []
#             ocr_result = []
#             vgg19_result = []


            cnt = 1

            for im in sorted(os.listdir(out_path)):
                pdf_name.append(pdf_file)
                file_name.append(cnt)
                #print(cnt)
                print(cnt,'  ',im)
                cnt = cnt + 1
                img = image.load_img(os.path.join(out_path,im),
                                 target_size=(224,224))
                #plt.imshow(img)
                #plt.show()

                x = image.img_to_array(img)
                x = x/255
                x = np.expand_dims(x,axis=0)
                images = np.vstack([x])
                val = inception_model.predict(images)
                #print(val)

                a=np.argmax(val,axis=1)
                output_inception = dic[a[0]]
                #print(output_inception)
                #file_name.append(im)

                #image_read = cv2.imread(os.path.join(in_path,im),0)
                pil_read = Image.open(os.path.join(out_path,im))
                pil_read_gray = ImageOps.grayscale(pil_read)
                img_rgb = np.array(pil_read_gray)


                _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

                nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
                sizes = stats[1:, -1] #get CC_STAT_AREA component
                img2 = np.zeros((labels.shape), np.uint8)

                for i in range(0, nlabels - 1):
                    if sizes[i] >= 50:   #filter small dotted regions
                        img2[labels == i + 1] = 255

                res = cv2.bitwise_not(img2)
            #     data = Image.fromarray(res)
            #     enh_con = ImageEnhance.Contrast(data)
            #     contrast = 3.0
            #     image_contrasted = enh_con.enhance(contrast)
                image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
                image_data_lower = " ".join(image_data.splitlines()).lower()

                stats = ImageStat.Stat(pil_read)
                #print(stats)

                y = []
                for band,name in enumerate(pil_read.getbands()):
                    x=sum(stats.extrema[band])
                    y.append(x)
                if sum(y)==765:
                    #print('good')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray)
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)

                else:
                    #print('dark')
                    image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
                    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                    image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
                    #print(image_data_1_lower)



                if output_inception == 'Blank':
                    inception_result.append(output_inception)
                else:


                    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                        inception_result.append('New Document')
                    else:
                        inception_result.append('Not Blank')


                val_1 = vgg_model.predict(images)
                #print(val_1)

                a_1=np.argmax(val_1,axis=1)
                output_vgg = dic[a_1[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg == 'Blank':
                    vgg_result.append(output_vgg)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg > 80 or fuzz_ratio_vgg_1 > 80:
                        vgg_result.append('New Document')
                    else:
                        vgg_result.append('Not Blank')


                val_2 = vgg19_model.predict(images)
                #print(val_1)

                a_2=np.argmax(val_2,axis=1)
                output_vgg19 = dic[a_2[0]]
                #print(output_vgg)
                #file_name.append(im)
                #vgg_result.append(output_vgg)

                if output_vgg19 == 'Blank':
                    vgg19_result.append(output_vgg19)
                else:
                    #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                    #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                    #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                    fuzz_ratio_vgg19 = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_vgg19_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio_vgg19 > 80 or fuzz_ratio_vgg19_1 > 80:
                        vgg19_result.append('New Document')
                    else:
                        vgg19_result.append('Not Blank')



                fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                    ocr_result.append('New Document')

                elif len(image_data_1_lower) < 15:
                    ocr_result.append('Blank')

                else:
                    ocr_result.append('Not Blank')


df = pd.DataFrame({'File_Name':pdf_name,'Page_Number':file_name,'Inception_result':inception_result,
                               'VGG_result':vgg_result,'VGG19_result':vgg19_result,
                               'OCR_Result':ocr_result})
#display(df)
df['Confidence']=(df['Inception_result']==df['VGG_result'])&(df['Inception_result']==df['VGG19_result'])&(df['Inception_result']==df['OCR_Result'])

df['status'] = df['Inception_result']
df['Confidence'] = df['Confidence'].replace(True,'Yes')
df['Confidence'] = df['Confidence'].replace(False,'No')

pd.set_option('display.max_rows',1000)
display(df)

final_df = df[['File_Name','Page_Number','status','Confidence']]
display(final_df)

final_df.to_excel(os.path.join(excel_path,
                               str(folder_path_name)+'.xlsx'),index=False)            

end = time.time()

print(end - start)

pdf
1507-1513.PDF
No of Images:  4
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
Batch 17 (1).Pdf
No of Images:  24
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg


,File_Name,Page_Number,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,1507-1513.PDF,1,New Document,New Document,New Document,New Document,Yes,New Document
1,1507-1513.PDF,2,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
2,1507-1513.PDF,3,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
3,1507-1513.PDF,4,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
4,Batch 17 (1).Pdf,1,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
5,Batch 17 (1).Pdf,2,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
6,Batch 17 (1).Pdf,3,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
7,Batch 17 (1).Pdf,4,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
8,Batch 17 (1).Pdf,5,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
9,Batch 17 (1).Pdf,6,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


,File_Name,Page_Number,status,Confidence
0,1507-1513.PDF,1,New Document,Yes
1,1507-1513.PDF,2,Not Blank,Yes
2,1507-1513.PDF,3,Not Blank,Yes
3,1507-1513.PDF,4,Not Blank,Yes
4,Batch 17 (1).Pdf,1,Not Blank,Yes
5,Batch 17 (1).Pdf,2,Not Blank,Yes
6,Batch 17 (1).Pdf,3,Not Blank,Yes
7,Batch 17 (1).Pdf,4,Not Blank,Yes
8,Batch 17 (1).Pdf,5,Not Blank,Yes
9,Batch 17 (1).Pdf,6,Not Blank,Yes


NameError: name 'start' is not defined

# Multiple folder model

In [21]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
import zipfile
from pathlib import Path

from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

start = time.time()

file_path = '/home/vimalkumar/Documents/stewart/2/stewart_test'

folder_path_name = os.path.basename(file_path)
print(folder_path_name)



images_path = os.path.join('/home/vimalkumar/Documents/stewart/2/images',folder_path_name)
#excel_path = '/home/vimalkumar/Documents/stewart/excel_output'
excel_path = '/home/vimalkumar/Documents/stewart/2/stewart_test'

os.makedirs(images_path)
#os.makedirs(os.path.join(excel_path,os.path.splitext(zip_file)[0]))



for folder in sorted(os.listdir(file_path)):
    print(folder)
    
    pdf_name = []
    file_name = []
    inception_result = []
    vgg_result = []
    ocr_result = []
    vgg19_result = []
    
    for pdf_file in sorted(os.listdir(os.path.join(file_path,folder))):
        pdf_file_extension = pdf_file.lower()
        if pdf_file_extension.endswith('.pdf'):
            print(pdf_file)
            in_path = os.path.join(file_path,folder,pdf_file)
            out_path = os.path.join(images_path,folder,
                                    str(os.path.splitext(pdf_file)[0]))

            os.makedirs(out_path)

            file_size=Path(os.path.join(file_path,folder,pdf_file)).stat().st_size

            if file_size != 0:
                doc = fitz.open(in_path)

                print('No of Images: ',len(doc))



                for i in range(len(doc)):
                    doc = fitz.open(in_path)
                    page = doc.load_page(i)  # number of page
                    pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
                    f_name = str(i+1000 + 1)       
                    pix.pil_save(os.path.join(out_path,f_name+'.jpeg'))
                    
                dic = {0:'Blank',1:'Not Blank'}

                inception_model = keras.models.load_model('stewart_inception_model.h5')
                vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')
                vgg19_model = keras.models.load_model('stewart_vgg19_model_1.h5')


    #             pdf_name = []
    #             file_name = []
    #             inception_result = []
    #             vgg_result = []
    #             ocr_result = []
    #             vgg19_result = []


                cnt = 1

                for im in sorted(os.listdir(out_path)):
                    pdf_name.append(pdf_file)
                    file_name.append(cnt)
                    #print(cnt)
                    print(cnt,'  ',im)
                    cnt = cnt + 1
                    img = image.load_img(os.path.join(out_path,im),
                                     target_size=(224,224))
                    #plt.imshow(img)
                    #plt.show()

                    x = image.img_to_array(img)
                    x = x/255
                    x = np.expand_dims(x,axis=0)
                    images = np.vstack([x])
                    val = inception_model.predict(images)
                    #print(val)

                    a=np.argmax(val,axis=1)
                    output_inception = dic[a[0]]
                    #print(output_inception)
                    #file_name.append(im)

                    #image_read = cv2.imread(os.path.join(in_path,im),0)
                    pil_read = Image.open(os.path.join(out_path,im))
                    pil_read_gray = ImageOps.grayscale(pil_read)
                    img_rgb = np.array(pil_read_gray)


                    _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

                    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
                    sizes = stats[1:, -1] #get CC_STAT_AREA component
                    img2 = np.zeros((labels.shape), np.uint8)

                    for i in range(0, nlabels - 1):
                        if sizes[i] >= 50:   #filter small dotted regions
                            img2[labels == i + 1] = 255

                    res = cv2.bitwise_not(img2)
                #     data = Image.fromarray(res)
                #     enh_con = ImageEnhance.Contrast(data)
                #     contrast = 3.0
                #     image_contrasted = enh_con.enhance(contrast)
                    image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
                    image_data_lower = " ".join(image_data.splitlines()).lower()

                    stats = ImageStat.Stat(pil_read)
                    #print(stats)

                    y = []
                    for band,name in enumerate(pil_read.getbands()):
                        x=sum(stats.extrema[band])
                        y.append(x)
                    if sum(y)==765:
                        #print('good')
                        image_data_1 = pytesseract.image_to_string(pil_read_gray)
                        image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                        image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)

                    else:
                        #print('dark')
                        image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
                        image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
                        image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
                        #print(image_data_1_lower)



                    if output_inception == 'Blank':
                        inception_result.append(output_inception)
                    else:


                        fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                        fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                        if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                            inception_result.append('New Document')
                        else:
                            inception_result.append('Not Blank')


                    val_1 = vgg_model.predict(images)
                    #print(val_1)

                    a_1=np.argmax(val_1,axis=1)
                    output_vgg = dic[a_1[0]]
                    #print(output_vgg)
                    #file_name.append(im)
                    #vgg_result.append(output_vgg)

                    if output_vgg == 'Blank':
                        vgg_result.append(output_vgg)
                    else:
                        #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                        #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                        #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                        fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
                        fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                        if fuzz_ratio_vgg > 80 or fuzz_ratio_vgg_1 > 80:
                            vgg_result.append('New Document')
                        else:
                            vgg_result.append('Not Blank')


                    val_2 = vgg19_model.predict(images)
                    #print(val_1)

                    a_2=np.argmax(val_2,axis=1)
                    output_vgg19 = dic[a_2[0]]
                    #print(output_vgg)
                    #file_name.append(im)
                    #vgg_result.append(output_vgg)

                    if output_vgg19 == 'Blank':
                        vgg19_result.append(output_vgg19)
                    else:
                        #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
                        #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
                        #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

                        fuzz_ratio_vgg19 = fuzz.partial_ratio(image_data_lower,'new document to follow')
                        fuzz_ratio_vgg19_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                        if fuzz_ratio_vgg19 > 80 or fuzz_ratio_vgg19_1 > 80:
                            vgg19_result.append('New Document')
                        else:
                            vgg19_result.append('Not Blank')



                    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
                    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

                    if fuzz_ratio > 80 or fuzz_ratio_1 > 80:
                        ocr_result.append('New Document')

                    elif len(image_data_1_lower) < 15:
                        ocr_result.append('Blank')

                    else:
                        ocr_result.append('Not Blank')


    df = pd.DataFrame({'File_Name':pdf_name,'Page_Number':file_name,'Inception_result':inception_result,
                                   'VGG_result':vgg_result,'VGG19_result':vgg19_result,
                                   'OCR_Result':ocr_result})
    #display(df)
    df['Confidence']=(df['Inception_result']==df['VGG_result'])&(df['Inception_result']==df['VGG19_result'])&(df['Inception_result']==df['OCR_Result'])

    df['status'] = df['Inception_result']
    df['Confidence'] = df['Confidence'].replace(True,'Yes')
    df['Confidence'] = df['Confidence'].replace(False,'No')

    pd.set_option('display.max_rows',1000)
    display(df)

    final_df = df[['File_Name','Page_Number','status','Confidence']]
    display(final_df)

    final_df.to_excel(os.path.join(excel_path,
                                   str(folder)+'.xlsx'),index=False)            

end = time.time()

print(end - start)
                    

stewart_test
box_1
1507-1513.pdf
No of Images:  4
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
Batch 17 (1).pdf
No of Images:  24
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg


,File_Name,Page_Number,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,1507-1513.pdf,1,New Document,New Document,New Document,New Document,Yes,New Document
1,1507-1513.pdf,2,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
2,1507-1513.pdf,3,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
3,1507-1513.pdf,4,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
4,Batch 17 (1).pdf,1,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
5,Batch 17 (1).pdf,2,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
6,Batch 17 (1).pdf,3,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
7,Batch 17 (1).pdf,4,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
8,Batch 17 (1).pdf,5,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
9,Batch 17 (1).pdf,6,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


,File_Name,Page_Number,status,Confidence
0,1507-1513.pdf,1,New Document,Yes
1,1507-1513.pdf,2,Not Blank,Yes
2,1507-1513.pdf,3,Not Blank,Yes
3,1507-1513.pdf,4,Not Blank,Yes
4,Batch 17 (1).pdf,1,Not Blank,Yes
5,Batch 17 (1).pdf,2,Not Blank,Yes
6,Batch 17 (1).pdf,3,Not Blank,Yes
7,Batch 17 (1).pdf,4,Not Blank,Yes
8,Batch 17 (1).pdf,5,Not Blank,Yes
9,Batch 17 (1).pdf,6,Not Blank,Yes


box_2
Batch 17 (1).pdf
No of Images:  24
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg


,File_Name,Page_Number,Inception_result,VGG_result,VGG19_result,OCR_Result,Confidence,status
0,Batch 17 (1).pdf,1,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
1,Batch 17 (1).pdf,2,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
2,Batch 17 (1).pdf,3,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
3,Batch 17 (1).pdf,4,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
4,Batch 17 (1).pdf,5,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
5,Batch 17 (1).pdf,6,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
6,Batch 17 (1).pdf,7,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
7,Batch 17 (1).pdf,8,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
8,Batch 17 (1).pdf,9,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank
9,Batch 17 (1).pdf,10,Not Blank,Not Blank,Not Blank,Not Blank,Yes,Not Blank


,File_Name,Page_Number,status,Confidence
0,Batch 17 (1).pdf,1,Not Blank,Yes
1,Batch 17 (1).pdf,2,Not Blank,Yes
2,Batch 17 (1).pdf,3,Not Blank,Yes
3,Batch 17 (1).pdf,4,Not Blank,Yes
4,Batch 17 (1).pdf,5,Not Blank,Yes
5,Batch 17 (1).pdf,6,Not Blank,Yes
6,Batch 17 (1).pdf,7,Not Blank,Yes
7,Batch 17 (1).pdf,8,Not Blank,Yes
8,Batch 17 (1).pdf,9,Not Blank,Yes
9,Batch 17 (1).pdf,10,Not Blank,Yes


143.90313267707825


In [1]:
import jovian

In [2]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Committed successfully! https://jovian.ai/vimalmadhan218-v2/stewart-2-0


'https://jovian.ai/vimalmadhan218-v2/stewart-2-0'